# Capstone Project

Pedagogical Objectives:

- Incorporate Bash, Python, SQL, Git into a single nose-to-tail project.

Students don't necessarily have the chops for a recommendation engine, but leave that as a stretch goal?

Old project:
 - rankine
 
New project ideas:
 - recommendation engine
 - maybe provide engine but require file parsing + database calls?

bash to marshal data files
python to clean data files
sql to store database for calls
python to hook up provided recommendation engine

(also check 498IM records)

datasets:
- soda (break into per-transaction files?)
    - generate review database as well so can see volume + tx + reviews
- IMDB (boring, already done, but could serve as inspiration)
    - https://github.com/lekhav/RecommendationSystem-IMDBdataset/blob/master/Recommendation%20System%20on%20IMDB%20Movies%20dataset.ipynb
- chinook, https://www.sqlitetutorial.net/sqlite-sample-database/
- City of Chicago data
    - https://data.cityofchicago.org/


In [8]:
import pandas as pd

invoice_df = pd.read_csv('invoice.csv')
item_df = pd.read_csv('item.csv')

In [9]:
invoice_df.head()

,Invoice_id,Date,Item_id,Vendor_id,Vendor_Name,Store_id,Store_Name,Address,City_Name,Zip_Code,County_id,County_Name,Bottles_Sold
0,INV-00013400001,2016-08-29,35918,297,Katou Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
1,INV-00013400002,2016-08-29,23828,297,Katou Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
2,INV-00013400003,2016-08-29,36908,300,Katsuragi Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1
3,INV-00013400004,2016-08-29,34359,35,Archer Inc.,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,8
4,INV-00013400005,2016-08-29,36903,300,Katsuragi Company,3882,Kwik Shop #579 / Davenport,2805 TELEGRAPH RD,DAVENPORT,52802,82,Scott,1


In [10]:
item_df.head()

,Item_id,Item_Description,Category,Pack,Bottle_Volume_ml,Bottle_Cost,Bottle_Retail_Price
0,101,Kanna's Grapefruit Soda,Grapefruit Soda,6,750,4.32,6.48
1,102,Alphonse's Fruit Punch,Fruit Punch,6,800,3.33,5.00
2,103,Yummy Surstromming Juice,Cream Soda,1,750,10.30,20.10
3,107,Tamaki's Watermelon Cream,Cream Soda,6,750,4.44,6.66
4,108,Kaori's Grapefruit Soda,Grapefruit Soda,6,750,3.12,4.68


In [11]:
merged_df = invoice_df.merge(item_df,how='left',left_on='Item_id',right_on='Item_id')

In [12]:
review_dict = {
    'Customer_id': ('1','2','3','4','5'),
    'Item_id': (101,102,103,107,108),
    'Review': (5,4,3,2,1)
}

In [13]:
pd.DataFrame.from_dict(review_dict)

,Customer_id,Item_id,Review
0,1,101,5
1,2,102,4
2,3,103,3
3,4,107,2
4,5,108,1


In [14]:
merged_df[merged_df['Bottle_Volume_ml'] == 0]

,Invoice_id,Date,Item_id,Vendor_id,Vendor_Name,Store_id,Store_Name,Address,City_Name,Zip_Code,County_id,County_Name,Bottles_Sold,Item_Description,Category,Pack,Bottle_Volume_ml,Bottle_Cost,Bottle_Retail_Price


In [15]:
import numpy as np

num_invoices = len(merged_df['Invoice_id'])

#num_customers = 8192
#customer_ids = np.random.randint(1000,10000,size=(num_customers,))

#num_reviews = 54321
ratings = np.random.randint(1,6,size=(num_invoices,))
#sorted(merged_df['Bottle_Cost'] / merged_df['Bottle_Volume_ml'],reverse=True)
ratings = np.random.randint(1,6,size=(num_invoices,))+10*merged_df['Bottle_Cost'] / merged_df['Bottle_Volume_ml']

#items = pd.unique(item_df['Item_id'])
ratings.astype(int)

0         2
1         1
2         3
3         4
4         2
         ..
930503    2
930504    2
930505    5
930506    2
930507    2
Length: 930508, dtype: int64

In [24]:
df = pd.Series(sorted(merged_df['Bottle_Cost'] / merged_df['Bottle_Volume_ml'],reverse=True))

np.max((5,)*len(df),df*100)

TypeError: 'Series' object cannot be interpreted as an integer

In [10]:
# Generate fake reviews
import sys
original_stdout = sys.stdout # Save a reference to the original standard output

review_dict = {
    'Review_id': range(num_reviews),
    'Customer_id': np.random.choice(customer_ids,size=num_invoices,replace=True),
    'Item_id': np.random.choice(items,size=num_invoices,replace=True),
    'Rating': ratings
}

reviews = pd.DataFrame.from_dict(review_dict)

for review in reviews.iterrows():
    with open(f'./reviews/review{str(review[1]["Review_id"]).zfill(5)}.dat','w') as review_file:
        sys.stdout = review_file
        print(f'Customer ID: {review[1]["Customer_id"]}')
        print(f'Item ID: {review[1]["Item_id"]}')
        print(f'Product Rating: {review[1]["Rating"]}/5',end='')
        sys.stdout = original_stdout

NameError: name 'num_reviews' is not defined

In [53]:
type(review)

tuple

In [54]:
review[1]

Review_id      54320
Customer_id     9581
Item_id        10836
Rating             4
Name: 54320, dtype: int64

Rubric:

0. Report on structure (fields, etc.).
1. Load all data files into single database (using Python, can be SQL or Python).
2. Merge with review data.
3. Plug into (provided) recommendation engine.
4. Visualize results.

Invoice data anomalies (outliers) tagged for further investigation
Review data for recommendation engine

start with subset until you know you can load it because it will take a long time
you can load in Pandas or PySQL as a database
which fields should be integers and which should be strings?

In [46]:
import glob
files = glob.glob('./reviews/*.dat')[:1000]  # start with a subset

fields = ['Review_id','Customer_id','Item_id','Rating']
reviews = pd.DataFrame(columns=fields)

for f in files:
    with open(f,'r') as review_file:
        review_data = review_file.readlines()
        tmp_dict = {}
        tmp_dict['Review_id'] = str(f).split('/')[-1][6:-4]
        tmp_dict['Customer_id'] = review_data[0][12:].strip().strip()
        tmp_dict['Item_id'] = int(review_data[1][9:].strip())
        tmp_dict['Rating'] = int(review_data[2][16:17].strip())
        reviews = reviews.append(tmp_dict,ignore_index=True)

In [47]:
reviews.head()

,Review_id,Customer_id,Item_id,Rating
0,46548,6194,27098,1
1,35998,1301,28042,1
2,43216,8982,904943,5
3,10003,7244,43986,4
4,22714,8294,4032,3


In [48]:
merged_df = reviews.merge(item_df,how='left',left_on='Item_id',right_on='Item_id')

In [49]:
merged_df

,Review_id,Customer_id,Item_id,Rating,Item_Description,Category,Pack,Bottle_Volume_ml,Bottle_Cost,Bottle_Retail_Price
0,46548,6194,27098,1,L's Old Fashioned Cream Soda,Cream Soda,6,750.0,3.59,5.39
1,35998,1301,28042,1,Joseph's Vanilla Cream Soda,Cream Soda,24,375.0,2.24,3.36
2,43216,8982,904943,5,Allen's Fruit Punch,Fruit Punch,24,375.0,2.06,3.09
3,10003,7244,43986,4,Light's Energy Booster,Energy Drink,6,1750.0,3.13,4.70
4,22714,8294,4032,3,Byakuya's Ginger Beer,Ginger Ale,1,2250.0,7.63,11.44
...,...,...,...,...,...,...,...,...,...,...
995,04723,1164,987507,4,Kaori's Kola,Cola,24,200.0,2.16,3.24
996,49756,3156,34736,2,Taiga's Red Pop,Cherry Soda,12,750.0,3.19,4.79
997,24233,7038,327,5,Asuna's Tropical Fruit Punch,Fruit Punch,6,750.0,4.44,6.66
998,16956,9833,1799,1,Killua's Energy,Energy Drink,6,1750.0,4.04,6.06


In [50]:
from scipy import stats

In [61]:
m_cs = merged_df[merged_df['Category'] == 'Cream Soda']
m_rb = merged_df[merged_df['Category'] == 'Root Beer']
m_fp = merged_df[merged_df['Category'] == 'Fruit Punch']
m_ch = merged_df[merged_df['Category'] == 'Cherry Soda']


In [56]:
m_rb

,Review_id,Customer_id,Item_id,Rating,Item_Description,Category,Pack,Bottle_Volume_ml,Bottle_Cost,Bottle_Retail_Price
10,23154,2102,82626,2,Megumin's Root Beer,Root Beer,12,750.0,2.39,3.58
13,42456,8349,969703,2,Osamu's Root Beer,Root Beer,10,50.0,2.34,3.51
15,47308,1524,984568,4,Nozomu's Craft Root Beer,Root Beer,12,750.0,2.93,4.40
28,53744,7970,74089,3,Gohan's Craft Root Beer,Root Beer,12,750.0,3.90,5.85
32,00923,9740,75228,2,Jiraiya's Craft Root Beer,Root Beer,6,750.0,2.91,4.36
...,...,...,...,...,...,...,...,...,...,...
964,11288,4300,80296,4,Guts's Root Beer,Root Beer,6,750.0,3.26,4.89
969,14271,8674,80143,4,Grimmjow's Root Beer,Root Beer,6,750.0,3.16,4.74
972,27481,7679,69946,3,Osamu's Craft Root Beer,Root Beer,12,750.0,3.26,4.89
979,27326,4977,84616,1,Madoka's Craft Root Beer,Root Beer,12,750.0,2.06,3.25


In [62]:
stats.ttest_ind(m_cs['Rating'],m_ch['Rating'],equal_var=False)

Ttest_indResult(statistic=-1.3895478664861372, pvalue=0.1653423206557304)

In [63]:
m_ch

,Review_id,Customer_id,Item_id,Rating,Item_Description,Category,Pack,Bottle_Volume_ml,Bottle_Cost,Bottle_Retail_Price
5,02204,3916,967285,3,Seira's Black Cherry,Cherry Soda,6,750.0,3.30,4.95
8,07928,8541,64742,1,Yoko's Black Cherry,Cherry Soda,6,750.0,4.11,6.16
11,11643,3691,37886,4,Inori's Black Cherry,Cherry Soda,12,750.0,2.26,3.39
16,50651,7777,39943,1,Kenshin's Gourmet Black Cherry Soda,Cherry Soda,12,750.0,2.91,4.37
20,03444,9512,71731,2,Izuku's Family Secret Black Cherry,Cherry Soda,6,750.0,3.34,5.01
...,...,...,...,...,...,...,...,...,...,...
987,19456,4185,34030,5,Simon's Family Secret Black Cherry,Cherry Soda,12,750.0,3.12,4.68
989,52268,3019,34881,5,Tamaki's Red Pop,Cherry Soda,12,1000.0,3.78,5.67
993,45096,3953,34449,1,Stampede's Gourmet Black Cherry Soda,Cherry Soda,12,750.0,3.33,4.99
994,28981,5726,34458,1,Ruri's Black Cherry,Cherry Soda,6,1750.0,4.88,7.32


In [30]:
merged_df['Bottles_Sold'].std()

22.489171499256173

In [35]:
invoices_all = merged_df
avg = invoices_all['Bottles_Sold'].mean()
stdev = invoices_all['Bottles_Sold'].std()
invoices_all[invoices_all['Bottles_Sold'] > avg+7*stdev]

,Invoice_id,Date,Item_id,Vendor_id,Vendor_Name,Store_id,Store_Name,Address,City_Name,Zip_Code,County_id,County_Name,Bottles_Sold,Item_Description,Category,Pack,Bottle_Volume_ml,Bottle_Cost,Bottle_Retail_Price
24376,INV-01082500004,2016-10-20,11296,260,Inuyasha Brands,2635,Hy-Vee #4 / Davenport,4064 E 53RD ST,DAVENPORT,52807,82,Scott,275,Mayoi's Old Fashioned Cream Soda,Cream Soda,12,750,3.51,5.26
75977,INV-03293700001,2017-02-15,11788,115,Deviluke Inc.,3447,Sam's Club 6432 / Sioux City,4201 S. YORK ST.,SIOUX CITY,51106,97,Woodbury,197,Meliodas's Old Fashioned Cream Soda,Cream Soda,6,1750,2.87,4.41
136392,INV-05912100010,2017-07-05,11788,115,Deviluke Inc.,4202,Fareway Stores #829 / Sioux City,4267 SERGEANT RD,SIOUX CITY,51106,97,Woodbury,197,Meliodas's Old Fashioned Cream Soda,Cream Soda,6,1750,2.87,4.41
217649,S03381400037,2012-01-03,68037,260,Inuyasha Brands,3354,Sam's Club 8238 / Davenport,3845 ELMORE AVE.,DAVENPORT,52807,82,Scott,300,Dante's Craft Root Beer,Root Beer,12,1000,3.93,5.90
218412,S03438100006,2012-01-05,24456,65,Ayanami Brands,2554,Hy-Vee Food Store / Davenport,3019 ROCKINGHAM ROAD,DAVENPORT,52802,82,Scott,840,Asuna's Old Fashioned Cream Soda,Cream Soda,12,750,2.28,3.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927042,S33985000150,2016-08-17,65257,192,Gokou Co.,2625,Hy-Vee Convenience Store #2,3301 W KIMBERLY RD,DAVENPORT,52804,82,Scott,168,Gohan's Golden Ginger Ale,Ginger Ale,12,1000,3.69,5.54
928315,S34078200001,2016-08-24,19226,65,Ayanami Brands,3447,Sam's Club 6432 / Sioux City,4201 S. YORK ST.,SIOUX CITY,51106,97,Woodbury,288,Hideko's Watermelon Cream,Cream Soda,6,750,4.38,6.57
929531,S34103000080,2016-08-24,10808,260,Inuyasha Brands,2625,Hy-Vee Convenience Store #2,3301 W KIMBERLY RD,DAVENPORT,52804,82,Scott,360,Mako's Cream Soda,Cream Soda,12,1000,4.11,6.17
930191,S34113900073,2016-08-25,64870,421,Midoriya Inc.,2635,Hy-Vee #4 / Davenport,4064 E 53RD ST,DAVENPORT,52807,82,Scott,192,Saitama's Craft Root Beer,Root Beer,48,100,1.63,2.45


In [37]:
avg

9.8756474957765

In [38]:
stdev

22.489171499256173

In [2]:
# next TODO:
'''
/ put together clean dataframe with ratings for some invoices
/ build rec engine around stores
/ reconstruct ratings files
- send for review?
'''

#build a network of how every store how things rank

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
count_matrix = cv.fit_transform()

In [16]:
ratings

0         2.014343
1         1.014514
2         3.014571
3         4.109000
4         2.083000
            ...   
930503    2.050600
930504    2.044000
930505    5.043600
930506    2.043200
930507    2.048267
Length: 930508, dtype: float64